In [38]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all

In [40]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [41]:
#MNIST DATAset

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),# 받아온 데이터를 어떻게 변화시킬 것인가
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [42]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [56]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
        nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2)
                                    )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)   # 요 레이어의 초기화 진행
        torch.nn.init.xavier_uniform_(self.fc2.weight)   # 요 레이어의 초기화 진행
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [63]:
model = CNN().to(device)  # 저 device는 cuda 사용가능하면 cuda, 그렇지 않으면 

tempValue = torch.Tensor(1,1,28,28).to(device)    # 모델이 잘 완성되었는지 테스트 하기 위해 임의의 값을 모델에 넣음
print((model(tempValue).shape))

torch.Size([1, 10])


In [64]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [65]:
# training
total_batch = len(data_loader)
print("Learning Started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0      # 여기다가 하나씩 loss function의 값을 담을것임
    
    for X, Y in data_loader:    # X는 이미지, Y는 라벨임
        X = X.to(device)        # 토치 쿠다 텐서가 되어있어야 한다. 그래서 이 두줄 넣어주는 것이라고 함
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch    # cost를 계속 누적해나감
    
    print('[Epoch:{}] cost = {}' .format(epoch+1, avg_cost))  # 한 에폭이 끝날 때 마다 코스트 값 출력시킴
print('Learning Finished!!')

Learning Started. It takes sometime.
[Epoch:1] cost = 0.16517411172389984
[Epoch:2] cost = 0.040099579840898514
[Epoch:3] cost = 0.02978004701435566
[Epoch:4] cost = 0.02164291776716709
[Epoch:5] cost = 0.016912328079342842
[Epoch:6] cost = 0.01500448863953352
[Epoch:7] cost = 0.011082987301051617
[Epoch:8] cost = 0.01195457298308611
[Epoch:9] cost = 0.009030957706272602
[Epoch:10] cost = 0.009198788553476334
[Epoch:11] cost = 0.008080756291747093
[Epoch:12] cost = 0.0058960882015526295
[Epoch:13] cost = 0.007050026673823595
[Epoch:14] cost = 0.004915947560220957
[Epoch:15] cost = 0.007741077337414026
Learning Finished!!


In [70]:
with torch.no_grad():   # 테스트 할 때는 학습을 안할거니까 이렇게 적어준다고 받아들이자
    X_test = mnist_test.test_data.view(len(mnist_test ), 1, 28, 28).float().to(device)  # 테스트할 10000개의 데이터를 한번에 GPU에 넣었다
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9817999601364136
